In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split
% matplotlib inline
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from IPython.display import HTML
style = "<style>svg{width:70% !important;height:70% !important;}</style>"
HTML(style)

UsageError: Line magic function `%` not found.


Возвращаемся к дэйтингу, некий Анатолий зарегистрировался в приложении знакомств, некий алгоритм на основе машинного обучения порекомендовал Анатолию 30 фотографий из 100. Анатолий лайкнул 15 фотографий из 30 предложенных, в дальнейшем выяснилось, что Анатолий мог бы лайкнуть еще 30 фотографий из 70 непредложенных.

Будем считать, что  лайк Анатолия является положительным исходом, а отсутствие лайка - отрицательным. Соответственно предсказанный лайк, совпавший с реальным - True positive, предсказанный лайк, не завершившийся реальным лайком - False positive и т. д. Давайте оценим работу алгоритма с точки зрения метрик качества модели.

|  |Predicted likes|Predicted dislikes|
|:--|:-:|:-:|
|Real likes|15|30|
|Real dislikes|15|40|

Такой формат представления предсказания модели и реальных исходов называется confusion matrix. Глядя на такую таблицу, легко можно понять, где именно ошибается наша модель. А также рассчитать различные метрики качества модели.

## Проанализировав отношения Анатолия с алгоритмом машинного обучения можно сделать следующий вывод, precision равняется

In [3]:
tp = 15
tn = 40
fp = 15
fn = 30
precision = tp/(tp + fp)
recall = tp/(tp + fn)
print(f'точность:{precision}, полнота:{recall}, f-мера:{(2 * precision * recall) / (precision + recall)}')

точность:0.5, полнота:0.3333333333333333, f-мера:0.4


### Поупражняемся в расчётах precision. В задании даны переменные, содержащие информацию о песнях и артистах - X_train, y_train, X_test и y_test. Исходно в датасэтах содержались тексты песен, но Decision Tree работает с категориальными и числовыми переменными, а текст это... текст. Поэтому его необходимо преобразовать в понятную для модели форму. В данном случае для каждой песни просто посчитаны длина и количество некоторых знаков пунктуации.

Обучите модель на тренировочных данных, предскажите авторов для тестовых и поместите в переменную predictions. Затем посчитайте precision score на предсказаниях и y_test, укажите параметр average='micro', и сохраните результат в переменную precision.

Если он будет недостаточно высок, потюньте немного модель.

Исходные данные взяты отсюда, слегка процессированные можно взять здесь (исходные колонки типа жанра, года были выкинуты в задании)

In [59]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score

In [4]:
songs_df = pd.read_csv('data/songs.csv')
songs_df.head()

,song,year,artist,genre,lyrics,length,commas,exclamations,colons,hyphens
0,the-ownerz,2007,gangstarr,Hip-Hop,"""One-two..""\n""One-two, shots to the chin.. kno...",1300,24,5,0,11
1,dead-sound-of-misery,2006,blind-guardian,Metal,It's dark\nLet's do it now\nBring in the siren...,1566,0,0,0,0
2,modred-s-song,2006,blind-guardian,Metal,I've lost my battle before it starts\nmy first...,996,1,0,1,0
3,the-ninth-wave,2015,blind-guardian,Metal,As dark night embraces\nWe are the nation\nI w...,2868,5,0,0,0
4,skills,2014,gangstarr,Hip-Hop,"Skills, skills, skills\n""My Microphone""\n""It's...",3351,22,0,0,3


In [35]:
def artist_to_num(key):
    d = {'abba': 1,
         'gangstarr': 2,
         'blind-guardian': 3}
    return d.get(key)

print(artist_to_num('gangstarr'))
songs_df.artist.value_counts()
songs_df['artist_num'] = songs_df.artist.apply(lambda x: artist_to_num(x))
songs_df.head()

2


,song,year,artist,genre,lyrics,length,commas,exclamations,colons,hyphens,artist_num
0,the-ownerz,2007,gangstarr,Hip-Hop,"""One-two..""\n""One-two, shots to the chin.. kno...",1300,24,5,0,11,2
1,dead-sound-of-misery,2006,blind-guardian,Metal,It's dark\nLet's do it now\nBring in the siren...,1566,0,0,0,0,3
2,modred-s-song,2006,blind-guardian,Metal,I've lost my battle before it starts\nmy first...,996,1,0,1,0,3
3,the-ninth-wave,2015,blind-guardian,Metal,As dark night embraces\nWe are the nation\nI w...,2868,5,0,0,0,3
4,skills,2014,gangstarr,Hip-Hop,"Skills, skills, skills\n""My Microphone""\n""It's...",3351,22,0,0,3,2


In [36]:
songs_df.shape

(360, 11)

In [51]:
X = songs_df.drop(['song', 'genre', 'lyrics', 'artist'], axis='columns')
X = pd.get_dummies(X)
y = songs_df.artist_num
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [52]:
X.dtypes

year            int64
length          int64
commas          int64
exclamations    int64
colons          int64
hyphens         int64
artist_num      int64
dtype: object

In [53]:
X.head()

,year,length,commas,exclamations,colons,hyphens,artist_num
0,2007,1300,24,5,0,11,2
1,2006,1566,0,0,0,0,3
2,2006,996,1,0,1,0,3
3,2015,2868,5,0,0,0,3
4,2014,3351,22,0,0,3,2


In [66]:
clr = tree.DecisionTreeClassifier(max_depth=100)
clr.fit(X_train, y_train)
precision_score(y_test, clr.predict(X_test), average='micro')

DecisionTreeClassifier(max_depth=100)

# В итоге мой ответ:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
clr = DecisionTreeClassifier(max_depth=100)
clr.fit(X_train, y_train)
predictions = clr.predict(X_test)
precision = precision_score(y_test, predictions , average='micro')